# Company Brochure Generation

In [ ]:
# Author: Sushant Mittal
# LinkedIn: https://www.linkedin.com/in/sushantmittal-sm/
print("Hello :)")

In [ ]:
# Loading the libraries:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

import sys
sys.path.append('../')
from scraper import fetch_website_contents, fetch_website_links

In [ ]:
# Connecting with Ollama and testing Connection:
load_dotenv()
ollama = OpenAI(
    api_key="OLLAMA",
    base_url="http://localhost:11434/v1"
)

openr = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

MODEL="qwen2.5"
response = ollama.chat.completions.create(
    model=MODEL,
    messages=[
        {"role":"system", "content":"You are a helpful assistant."},
        {"role":"user", "content":"Which model are you?"},
    ]
)

response.choices[0].message.content

In [ ]:
# Getting the links for the website that are appropriate for Brochure:

link_system_prompt = """
You are a expert in selecting relevant information required to be published in the Brochure. Provided the URLs from the website you pick the ones that contains most relevant information for Brochure creation.
You provide the output of selected URLs along with assigned name in JSON format as following:
[
    {"name": "about page", "url": "https://full.url/goes/here/about"},
    {"name": "careers page", "url": "https://another.full.url/careers"}
]
"""

def get_link_user_prompt(url):
    user_prompt = f"""
    Here are the links on the website {url}:
    Kindly think and decide which of them can contain relevant Brochure information. Reply the relevant ones with full URLs in JSON Format. 
    Don't include links that contain private information, emails and contact information. The output should strictly return a JSON in expected format with no additional information, so that it can be converted to python dictionary with json.loads().
    
    Links:
    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt


def get_relevant_links_ollama(url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_link_user_prompt(url)},
        ],
        response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

def get_relevant_links_openr(url):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_link_user_prompt(url)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

In [ ]:
# url=input("Enter URL:")
# print(get_relevant_links_openr(url))

In [ ]:
# Brochure Generation Code

brochure_system_prompt = """
You are an expert in designing Colorful and Markdown format Brochures for the company given the relevant content from various links/urls from the companies website.
"""

def get_brochure_user_prompt(url):
    brochure_user_prompt = """
    Here's the content from the website for various pages. Design the company Brochure using this information:

    Content:
    """
    webpage_contents = ["## "+link['name']+":\n"+fetch_website_contents(link['url'])+"\n\n" for link in json.loads(get_relevant_links_openr(url))]
    brochure_user_prompt += "\n".join(webpage_contents)
    return brochure_user_prompt

def get_brochure(url):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":brochure_system_prompt},
            {"role":"user", "content":get_brochure_user_prompt(url)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content

In [ ]:
brochure = get_brochure("https://www.huggingface.co")
display(Markdown(brochure))

In [ ]:
# Brochure Translation (English to Spanish)
translation_system_prompt = """
You're a translator that translates the given information in English to Spanish, maintaining the input format. For example: If Markdown format data is provided, you return the translated data in same Mardown format.
"""

def get_translation_user_prompt(brochure):
    translation_user_prompt = """
    Translate the following Brochure into Spanish:

    """
    translation_user_prompt += brochure
    return translation_user_prompt

def get_spanish_brochure(brochure):
    response = openr.chat.completions.create(
        model="nvidia/nemotron-3-nano-30b-a3b:free",
        messages=[
            {"role":"system", "content":translation_system_prompt},
            {"role":"user", "content":get_translation_user_prompt(brochure)},
        ],
        #response_format={"type":"json_schema"}
    )
    return response.choices[0].message.content


In [ ]:
spanish_brochure = get_spanish_brochure(brochure)
display(Markdown(spanish_brochure))